In [ ]:
!pip install gensim

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2600ab4a36dfa91eeb36d6c5b49f3d6a158a44ee67b3240174fa3840da00f99f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
pip install scikit-learn nltk

In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
import nltk
import gensim
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim import corpora
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
from rouge_score import rouge_scorer

In [ ]:
# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# Load dataset
# Load dataset, trying different encodings
try:
    df = pd.read_csv('/content/dam.csv', encoding='utf-8')  # Try UTF-8 first
except UnicodeDecodeError:
    df = pd.read_csv('/content/dam.csv', encoding='latin-1')  # Try Latin-1 if UTF-8 fails
df

,SNO,NAME OF DAM,District,Taluk,Block,Nearest city/town,Longitude of dam,Latitude of dam,Year of Completion,River,...,Effective storage capacity \n(103 m 3 ),MWL (m),FRL (m),Designed spillway capacity (m3 /sec),Spillway Type,Purpose,Command Area (Acres),Hydro Electric Power (MW),Latitude,Longitude
0,1,Periyar (Mullai Periyar),Idukki,Peermedu,Azhutha,Kumily,"09°31'44""","77°08'40""",1895,Mullai Periyar,...,299310.69,873.20,872.285,3454.000,Vertical Lift Shutters 10 Nos.\nRadial shutter...,I/H,212758,3x42\n1x35,9.528889,77.144444
1,2,Pechipparai,Kanyakumari,NaN,Thiruvattar,Kuzhithurai,"08°27'04""","77°18'30""",1906,Kodaiyar,...,123107.80,92.05,92.050,1104.000,NaN,I,38850.24 Combined,NaN,8.451111,77.308333
2,3,Willingdon,Villupuram,Titakudi,Mangalore,Thittagudi,"11°25'33""","79°06'16""",1923,Periyar odai,...,60010.00,75.70,72.540,384.000,Controlled Spillway,I,27675,NaN,11.425833,79.104444
3,4,Glenmorgan,Nilgiris,Ooty,Ooty,Ooty,"11°30'11""","76°35'27""",1930,Glenmorgan stream,...,NaN,NaN,2036.500,46.000,Uncontrolled,H,NaN,Diversion Dam,11.503056,76.590833
4,5,Mettur,Salem,Mettur,NaN,Salem,"11°48'11""","77°48'24""",1934,Cauvery,...,2645201.00,242.62,240.790,12904.800,Controlled spillway,I/H/S,1650000,240,11.803056,77.806667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,116,Andiappanur Odai,Vellore,Vanniyambadi,Thirupathur,Vellore,"12°30'34""","78°42'57""",2007,Pambar,...,3175.00,500.50,499.000,422.680,Uncontrolled Spillway,I,5025,NaN,12.509444,78.715833
116,117,Vandal Odai,Tirunelveli,Ambasamudram,Cheranmahadevi,Ambasamudram,"08°41'06""","77°26'15""",2007,Vandal Odai,...,1641.40,117.00,116.000,126.680,NaN,I,920.33,NaN,8.685000,77.437500
117,118,Sirumalaiyar Odai,Dindigul,Nila kottai,Nilakottai,Nilakottai,"10°17'36""","77°49'02""",2009,Sirumalaiyar,...,438.65,277.30,275.800,166.105,Uncontrolled,I,127.78,NaN,10.293333,77.817222
118,119,Kuppanatham,Thiruvannamalai,Chengam,Chengam,Thiruvannamalai,"12°25'11""","78°46'23""",2009,Cheyyar,...,19221.00,350.00,350.000,1359.200,Controlled Spillway,I,9812,NaN,12.419722,78.773056


In [ ]:
df.head()

,SNO,NAME OF DAM,District,Taluk,Block,Nearest city/town,Longitude of dam,Latitude of dam,Year of Completion,River,...,Effective storage capacity \n(103 m 3 ),MWL (m),FRL (m),Designed spillway capacity (m3 /sec),Spillway Type,Purpose,Command Area (Acres),Hydro Electric Power (MW),Latitude,Longitude
0,1,Periyar (Mullai Periyar),Idukki,Peermedu,Azhutha,Kumily,"09°31'44""","77°08'40""",1895,Mullai Periyar,...,299310.69,873.20,872.285,3454.0,Vertical Lift Shutters 10 Nos.\nRadial shutter...,I/H,212758,3x42\n1x35,9.528889,77.144444
1,2,Pechipparai,Kanyakumari,NaN,Thiruvattar,Kuzhithurai,"08°27'04""","77°18'30""",1906,Kodaiyar,...,123107.80,92.05,92.050,1104.0,NaN,I,38850.24 Combined,NaN,8.451111,77.308333
2,3,Willingdon,Villupuram,Titakudi,Mangalore,Thittagudi,"11°25'33""","79°06'16""",1923,Periyar odai,...,60010.00,75.70,72.540,384.0,Controlled Spillway,I,27675,NaN,11.425833,79.104444
3,4,Glenmorgan,Nilgiris,Ooty,Ooty,Ooty,"11°30'11""","76°35'27""",1930,Glenmorgan stream,...,NaN,NaN,2036.500,46.0,Uncontrolled,H,NaN,Diversion Dam,11.503056,76.590833
4,5,Mettur,Salem,Mettur,NaN,Salem,"11°48'11""","77°48'24""",1934,Cauvery,...,2645201.00,242.62,240.790,12904.8,Controlled spillway,I/H/S,1650000,240,11.803056,77.806667


In [ ]:
df.columns

Index(['SNO', 'NAME OF DAM', 'District', 'Taluk', 'Block', 'Nearest city/town',
       'Longitude of dam', 'Latitude of dam ', 'Year of Completion', 'River',
       'Tributary', 'Basin', 'Sub Basin', 'Seismic zone', 'Type of Dam',
       'Catchment Area (Sq.Km)', 'Length of dam (m)',
       'Height above lowest foundation (m)',
       'Gross storage capacity  (10 3m3)', 'Classification of Dams as per BIS',
       'Volume content of dam (10 3m3)', 'Water Spread area at  FRL (Sq.Km)',
       'Effective storage capacity \n(103 m 3 )', 'MWL (m)', 'FRL (m)',
       'Designed spillway capacity (m3  /sec)', 'Spillway Type', 'Purpose',
       'Command Area (Acres)', 'Hydro Electric Power (MW)', 'Latitude',
       'Longitude'],
      dtype='object')

In [ ]:
# Define the generate_summary function
def generate_summary(text, n):
    # Tokenize the text into individual sentences
    sentences = sent_tokenize(text)

    # Tokenize each sentence into individual words and remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word.lower() for word in word_tokenize(text) if word.lower() not in stop_words and word.isalnum()]

    # Compute the frequency of each word
    word_freq = Counter(words)

    # Compute the score for each sentence based on the frequency of its words
    sentence_scores = {}
    for sentence in sentences:
        sentence_words = [word.lower() for word in word_tokenize(sentence) if word.lower() not in stop_words and word.isalnum()]
        sentence_score = sum([word_freq[word] for word in sentence_words])
        if len(sentence_words) < 20:  # You can adjust this threshold as needed
            sentence_scores[sentence] = sentence_score

    # Select the top n sentences with the highest scores
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:n]
    summary = ' '.join(summary_sentences)

    return summary


In [ ]:
reindexed_data = df['NAME OF DAM']
reindexed_data.index = df['District']

In [ ]:
grouped = reindexed_data.groupby(reindexed_data.index).apply(' '.join).reset_index(name='NAME OF DAM')


In [ ]:
# Apply the summarization function to each group
grouped['summary'] = grouped['NAME OF DAM'].apply(lambda x: generate_summary(x, 5))

In [ ]:
# Get the most recent 5 dates
recent_dates = grouped['NAME OF DAM'].drop_duplicates().sort_values(ascending=False).head(5)

In [ ]:
# Display summaries with additional debugging information
for date, group in grouped.groupby('NAME OF DAM'):
    summary = group['summary'].iloc[0]
    print(f"Headlines for {date}: {group['District'].iloc[0]}")
    print("-" * 5)


Headlines for Aliyar Pillur Upper Aliyar Sholayar UpperNirarWeir LowerNirar Kadambarai Vandal weir: Coimbatore
-----
Headlines for Amaravathi Thirumurthi Uppar(Erode) Vattamalaikarai Odai Nallathangal Odai: Tirupur
-----
Headlines for Bhavanisagar Gunderipallam Varattupallam Perumpallam Noyyal Orathupalayam Kodaganar: Erode
-----
Headlines for Chinnar Thumbalahalli Kesarigullihalla Vaniar Nagavathi Thoppaiyar Kelavarapalli Varattar Vallimadurai: Dharmapuri
-----
Headlines for Glenmorgan Pykara Mukurthi Maravakandy Moyar forebay Avalanche Emerald Kundahpallam Sandynallah Bungihallahbund Pegumbahallah Forebay Upper Bhavani Upper Bhavani pumping weir Parson Valley Nirallapallam Porthimund Western Catchment Weir-I Western Catchment Weir-II Western Catchment Weir-III East Varahapallam weir West Varahapallam weir Pykara New Forebay: Nilgiris
-----
Headlines for Krishnagiri Pambar Sulagirichinnar: Krishnagiri
-----
Headlines for Kuthiraiyar Parappalar Palar Poranthalar Varadhamanadhi Nanganji

In [ ]:
def generate_summary(text, n):
    # Tokenize the text into individual sentences
    sentences = sent_tokenize(text)

    # Create the TF-IDF matrix
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Compute the cosine similarity between each sentence and the document
    cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix).mean(axis=1)

    # Select the top n sentences with the highest scores
    summary_sentences = nlargest(n, range(len(cosine_similarities)), key=cosine_similarities.__getitem__)

    # Join the selected sentences to form the summary
    summary_tfidf = ' '.join([sentences[i] for i in sorted(summary_sentences)])

    return summary_tfidf

In [ ]:
# Apply the summarization function to each row in the DataFrame
df['NAME OF DAM'] = df['District'].apply(lambda x: generate_summary(x, 5))  # Set to 5 sentences


In [ ]:
#Display the summaries
for index, row in df.iterrows():
    print(f"Dams: {row['NAME OF DAM']}")
    print(f"District: {row['District']}\n")

Dams: Idukki
District: Idukki

Dams: Kanyakumari
District: Kanyakumari

Dams: Villupuram
District: Villupuram

Dams: Nilgiris
District: Nilgiris

Dams: Salem
District: Salem

Dams: Nilgiris
District: Nilgiris

Dams: Nilgiris
District: Nilgiris

Dams: Thirunelveli
District: Thirunelveli

Dams: Thirunelveli
District: Thirunelveli

Dams: Tiruvallore
District: Tiruvallore

Dams: Nilgiris
District: Nilgiris

Dams: Nilgiris
District: Nilgiris

Dams: Kanyakumari
District: Kanyakumari

Dams: Erode
District: Erode

Dams: Tirupur
District: Tirupur

Dams: Krishnagiri
District: Krishnagiri

Dams: Tirunelveli
District: Tirunelveli

Dams: Theni
District: Theni

Dams: Tiruvannamalai
District: Tiruvannamalai

Dams: Theni
District: Theni

Dams: Villupuram
District: Villupuram

Dams: Nilgiris
District: Nilgiris

Dams: Nilgiris
District: Nilgiris

Dams: Nilgiris
District: Nilgiris

Dams: Coimbatore
District: Coimbatore

Dams: Nilgiris
District: Nilgiris

Dams: Nilgiris
District: Nilgiris

Dams: Villupura

In [ ]:
# Define the evaluation function
def evaluate_rouge(reference_text, summary_text):
    rouge = Rouge()
    scores = rouge.get_scores(reference_text, summary_text)
    return scores[0]['rouge-1']['f']

In [ ]:
import pandas as pd

# Dummy function definition for evaluate_rouge
# You should replace this with the actual implementation
def evaluate_rouge(reference_summary, generated_summary):
    # This is a placeholder function. Replace with your ROUGE evaluation code.
    # Should return a dictionary, e.g., {"rouge-1": {"f1": 0.75}}
    return {"rouge-1": {"f1": 0.75}}

# Initialize a list to store ROUGE-1 F1 scores
rouge_f1_scores = []

# Assuming 'reference_summary' column exists in the dataframe
for index, row in df.iterrows():
    generated_summary = row['NAME OF DAM']
    reference_summary = row['District']  # Replace with actual reference column

    # Ensure reference_summary is a string
    if not isinstance(reference_summary, str):
        reference_summary = str(reference_summary)

    # Compute ROUGE scores
    rouge_score = evaluate_rouge(reference_summary, generated_summary)

    # Extract ROUGE-1 F1 score from the result
    rouge_1_f1_score = rouge_score.get('rouge-1', {}).get('f1', 0.0)

    # Append ROUGE-1 F1 score to the list
    rouge_f1_scores.append(rouge_1_f1_score)

# Calculate average ROUGE-1 F1 score
if rouge_f1_scores:  # Check if the list is not empty
    average_rouge_1_f1_score = sum(rouge_f1_scores) / len(rouge_f1_scores)
else:
    average_rouge_1_f1_score = 0.0

# Print the average ROUGE-1 F1 score
print(f"Average ROUGE-1 F1 Score for the dataset: {average_rouge_1_f1_score:.4f}")


Average ROUGE-1 F1 Score for the dataset: 0.7500
